# Downloading data from Google Drive

In [ ]:
#!pip install gdown


In [ ]:
#import gdown
# to download the data from Google Drive
# id = 1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi   ( saber_11_pro.csv)
#url = 'https://drive.google.com/uc?export=download&id=1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi'
#output = 'saber_11_pro.csv'
#gdown.download(url, output, quiet=False)


In [ ]:
#import gdown
# to download the data from Google Drive
# id = 10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu'
#output = 'saber_combined_all_fields.csv'
#gdown.download(url, output, quiet=False)

In [ ]:
#import gdown
# to download the data from Google Drive
# id = 1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV'
#output = 'saber_combined_preprocessed.csv'
#gdown.download(url, output, quiet=False)

# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import plotly.express as px
import plotly.graph_objs as go

from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score,  r2_score,  f1_score, classification_report,roc_curve,roc_auc_score


 
from sklearn import model_selection  # Kfold..
# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

import warnings

pd.set_option('display.max_rows', 200) # to show more rows.
warnings.simplefilter('ignore')



# DF, Dataset : Saber_Combined_Preprocessed

In [2]:
df = pd.read_csv('../input/saber-combined-preprocessed/saber_combined_preprocessed.csv')
df.head()

In [3]:
def nan_check(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    return missing_data

nan_check(df)

In [4]:
replace_dict = {'AGRICULTURAL, FORESTRY ENGINEERING':'AGRICULTURAL, FOREST ENGINEERING',
               'AGROINDUSTRIAL ENGINEERING, FOOD':'AGROINDUSTRIAL AND FOOD ENGINEERING',
               'TRAINING RELATED TO THE MILITARY OR POLICE FIELD':'MILITARY OR POLICE TRAINING',
               'NUTRITION AND DIET':'NUTRITION AND DIETETICS'}
df['your_undergraduate_core'] = df['your_undergraduate_core'].replace(replace_dict)

In [5]:
df['your_undergraduate_core'].value_counts()

In [6]:
ENGINEERING_dict = {"INDUSTRIAL ENGINEERING":'ENGINEERING',
"ENVIRONMENTAL, SANITARY ENGINEERING":'ENGINEERING',
"CIVIL ENGINEERING":'ENGINEERING',
"COMPUTER SYSTEMS, TELEMATICS ENGINEERING":'ENGINEERING',
"MECHANICAL ENGINEERING":'ENGINEERING',
"ELECTRONIC ENGINEERING, TELECOMMUNICATIONS":'ENGINEERING',
"CHEMICAL ENGINEERING":'ENGINEERING',
"MINING, METALLURGY ENGINEERING":'ENGINEERING',
"OTHER ENGINEERING":'ENGINEERING',
"ELECTRICAL ENGINEERING":'ENGINEERING',
"AGROINDUSTRIAL AND FOOD ENGINEERING":'ENGINEERING',
"AGRONOMIC, LIVESTOCK ENGINEERING":'ENGINEERING',
"ADMINISTRATIVE ENGINEERING":'ENGINEERING',
"BIOMEDICAL ENGINEERING":'ENGINEERING',
"AGRICULTURAL, FOREST ENGINEERING":'ENGINEERING',
}


In [7]:
df = df[df['your_undergraduate_core'] != 'UNCLASSIFIED']
df = df[df['your_undergraduate_core'] != 'UNKNOWN']
df['your_undergraduate_core'].value_counts()


In [8]:
df['your_undergraduate_core'] = df['your_undergraduate_core'].replace(ENGINEERING_dict)
df['your_undergraduate_core'].value_counts()

# Building the Recommendation System

In [9]:
recsys_df = df.copy()
recsys_df.head()

In [10]:
# Final features
features_train = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
      # target
     'your_undergraduate_core'
]


In [57]:
recsys = recsys_df[features_train]
recsys.head()

In [58]:
# Final features
features = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
]
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys[var].describe())
    print('-----')


In [59]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(output_distribution='normal')

In [60]:
recsys['score_language_saber_11'] = qt.fit_transform(recsys[['score_language_saber_11']].values.reshape(-1,1))
recsys['score_mathematics_saber_11'] = qt.fit_transform(recsys[['score_mathematics_saber_11']].values.reshape(-1,1))
recsys['score_biology_saber_11'] = qt.fit_transform(recsys[['score_biology_saber_11']].values.reshape(-1,1))
recsys['score_chemistry_saber_11'] = qt.fit_transform(recsys[['score_chemistry_saber_11']].values.reshape(-1,1))
recsys['score_physics_saber_11'] = qt.fit_transform(recsys[['score_physics_saber_11']].values.reshape(-1,1))
recsys['score_social_science_saber_11'] = qt.fit_transform(recsys[['score_social_science_saber_11']].values.reshape(-1,1))
recsys['score_philosophy_saber_11'] = qt.fit_transform(recsys[['score_philosophy_saber_11']].values.reshape(-1,1))
recsys['score_english_saber_11'] = qt.fit_transform(recsys[['score_english_saber_11']].values.reshape(-1,1))

In [61]:
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys[var].describe())
    print('-----')

In [62]:
#create a mapping from labels to a unique integer and vice versa for labelling and prediction later
labels = recsys['your_undergraduate_core'].unique()
i = 0
idx2class = {} 
class2idx = {}
for tp in labels:
    idx2class[i] = tp
    class2idx[tp] = i
    i += 1
idx2class

In [63]:
recsys['target_career'] = recsys['your_undergraduate_core'].replace(class2idx)

In [64]:
recsys.head()

In [65]:
# Final features
features_tr = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
      # target
     'target_career'
]

df1 = recsys[features_tr]
df1.head(2)

In [66]:
def stratified_split(df, target, val_percent=0.2):
    '''
    Function to split a dataframe into train and validation sets, while preserving the ratio of the labels in the target variable
    Inputs:
    - df, the dataframe
    - target, the target variable
    - val_percent, the percentage of validation samples, default 0.2
    Outputs:
    - train_idxs, the indices of the training dataset
    - val_idxs, the indices of the validation dataset
    '''
    classes=list(df[target].unique())
    train_idxs, val_idxs = [], []
    for c in classes:
        idx=list(df[df[target]==c].index)
        np.random.shuffle(idx)
        val_size=int(len(idx)*val_percent)
        val_idxs+=idx[:val_size]
        train_idxs+=idx[val_size:]
    return train_idxs, val_idxs

In [67]:
train_idxs, val_idxs = stratified_split(df1, 'target_career', val_percent=0.25)

val_idxs, test_idxs = stratified_split(df1[df1.index.isin(val_idxs)], 'target_career', val_percent=0.5)

In [68]:
def test_stratified(df, col):
    '''
    Analyzes the ratio of different classes in a categorical variable within a dataframe
    Inputs:
    - dataframe
    - categorical column to be analyzed
    Returns: None
    '''
    classes=list(df[col].unique())
    
    for c in classes:
        print(f'Proportion of records with {c}: {len(df[df[col]==c])*1./len(df):0.2} ({len(df[df[col]==c])} / {len(df)})')

In [69]:
print('---------- STRATIFIED SAMPLING REPORT ----------')
print('-------- Label proportions in FULL data --------')
test_stratified(df1, 'target_career')
print('-------- Label proportions in TRAIN data --------')
test_stratified(df1[df1.index.isin(train_idxs)], 'target_career')
print('------ Label proportions in VALIDATION data -----')
test_stratified(df1[df1.index.isin(val_idxs)], 'target_career')
print('-------- Label proportions in TEST data ---------')
test_stratified(df1[df1.index.isin(test_idxs)], 'target_career')

In [70]:
train_df = df1[df1.index.isin(train_idxs)]
X_train = df1[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_train = df1[['target_career']].values
print('Retrieved Training Data')
val_df = df1[df1.index.isin(val_idxs)]
X_val = val_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_val = val_df[['target_career']].values
print('Retrieved Validation Data')
test_df = df1[df1.index.isin(test_idxs)]
X_test = test_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_test = test_df[['target_career']].values
print('Retrieved Test Data')

In [ ]:
!pip install visualkeras

# Neural Network

In [53]:
# multi-class classification with Keras
import tensorflow as tf
import visualkeras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from tensorflow.keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [71]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
encoded_Y_val = encoder.transform(Y_val)
encoded_Y_test = encoder.transform(Y_test)


# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = tf.keras.utils.to_categorical(encoded_Y_train)
dummy_y_val = tf.keras.utils.to_categorical(encoded_Y_val)
dummy_y_test = tf.keras.utils.to_categorical(encoded_Y_test)

In [30]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=8, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(64, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','MeanSquaredError','AUC','top_k_categorical_accuracy'])
    return model


In [72]:
# create model
model = Sequential()
model.add(Dense(100, input_dim=8, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(44, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','MeanSquaredError','AUC','top_k_categorical_accuracy'])

In [73]:
#model = baseline_model()
model.summary()

In [74]:
visualkeras.layered_view(model)

In [75]:
%%time
history = model.fit(X_train,dummy_y_train, validation_data=(X_val,dummy_y_val),epochs= 20,batch_size=128,verbose=1)

In [76]:
print('----Model--------------------------------- \n')
print('------------\n')
print('Model Accuracy = {}'.format(history.history['accuracy'][np.argmin(history.history['loss'])]*100))
print('Model TOP 5 Accuracy = {}'.format(history.history['top_k_categorical_accuracy'][np.argmin(history.history['loss'])]*100))
print('Model AUC = {}'.format(history.history['auc'][np.argmin(history.history['loss'])]*100))
print('Model RMSE = {}'.format(history.history['mean_squared_error'][np.argmin(history.history['loss'])]))
print('Model loss-categorical_crossentropy  = {}'.format(history.history['loss'][np.argmin(history.history['loss'])]))

In [80]:
# Evaluation on Train
model.evaluate(X_train,dummy_y_train)

In [81]:
# Evaluation on Vali
model.evaluate(X_val,dummy_y_val)

In [82]:
# Evaluation on Test
model.evaluate(X_test,dummy_y_test)

In [83]:
plt.figure(figsize=(8,6))
plt.title('Accuracy scores')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])

plt.figure(figsize=(8,6))
plt.title('AUC value')
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.legend(['auc', 'val_auc'])

plt.figure(figsize=(8,6))
plt.title('top_k_categorical_accuracy value')
plt.plot(history.history['top_k_categorical_accuracy'])
plt.plot(history.history['val_top_k_categorical_accuracy'])
plt.legend(['top_k_categorical_accuracy', 'val_top_k_categorical_accuracy'])

plt.figure(figsize=(8,6))
plt.title('RMSE value')
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_mean_squared_error'])
plt.legend(['mean_squared_error', 'val_mean_squared_error'])

plt.figure(figsize=(8,6))
plt.title('Loss value')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [84]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(Y_test, y_pred_bool))